# Android II

## Quick Review
- Layout in XML
- Behavoir in Java
- Use Android Studio to create and work on projects  

## Multi Activity Apps
- Most apps don't just consist of one screen (activity)
- Every app has one MAIN activity, which the OS uses to launch the app
    - From the MAIN activity, other activities can be launched
    - They aren't launched directly, rather an application sends a message to the OS asking the activity to be launched

## Intents
- An intent is an object that holds all the information about the new activity to be opened
- Can be 
    - explicit - Directly naming the exact activity class to open
    - implicit - Describing some desired features, and letting the OS choose
- After construction, the new activity is started by passing the intent object to the `startActivity` method

## Explicit Intent
- There are numerous constructors for the Intent class, but for explicit intents, the following is often used
```java
Intent my_intent = new Intent(Context c, Class cls)
```
    - The context is the current state
        - `Activity` ineherits from `Context`, so normally the first argument is `this`
    - The class is the `Class` object representing the activity we wish to get
        - `ClassName.class`

## Passing and Getting Data from Intents
- The Intent we just used can launch a new screen, but not much else
- To pass data we have two options
    - `Intent.setData(URI)` - pass a URI to a file, etc. for the recieving activity to use (1 per intent)
    - `Intent.putExtra(Key,Value)` - pass any number of small pieces of data (many per intent)
- Inside the newly launched activity, the intent used to launch it can be retrieved with
```java
    Intent launcher = getIntent();
```

## Getting a Result from an Activity
- Many times a new screen is launched for the purpose of collecting some input from the user, and then acting on that input
- To tell Android this is what we are doing, start the second activity with 
```java
startActivityForResult(Intent intent, int requestCode)
```
- This will allow several methods to be used that handle a lot of things for us

## In the Recieving Activity
- In an activity that is meant to return a result, the following is done
    - Get and process the launching intent as before
    - Create a new intent to send the result back in
    - Call `setResult(resultCode, resultIntent)` to indicate if the activity was successful
        - resultCode is one of: RESULT_OK, RESULT_CANCELLED, or RESULT_FIRST_USER (static constants)
    - Call `finish()` to both send the intent back, as well as close the second activity

## In Originating Activity
- Implement the `onActivityResult` event handler, which takes the following parameters
    - `int requestCode` - The request code used to start the second activity, useful for figuring out what you are expecting back
    - `int resultCode` - The resultCode set with `setResult` in the second activity
    - `Intent data` - The result intent created and sent back from the second activity

## The Lifecycle of an App 
- Up until now we have been focused on the `onCreate` method of the Activity
    - This is the first method called when the activity starts
- There are many other methods we can use as handlers, which are triggered at various points in the activities life style
    - `onStart`
    - `onResume()`
    - `onRestart()`
    - `onPause()`
    - `onStop()`

## The Lifecycle of an App 
<img src="https://google-developer-training.gitbooks.io/android-developer-fundamentals-course-concepts/content/en/images/2_2_C_images/basic-lifecycle.png">
From https://google-developer-training.gitbooks.io/android-developer-fundamentals-course-concepts/content/en/Unit%201/22_c_the_activity_lifecycle_and_managing_state.html

## Starting an External Activity 
- Up until now we have be starting activities we have the code for and therefore can use the `.class` member
    - This isn't possible with an external application
- One solution is to use the `PackageManager` class to construct an intent meant to launch an App
```java
PackageManager manager = this.getPackageManager();
//You need to know the full name of package for the app you are launching
Intent i = manager.getLaunchIntentForPackage(
                        "com.cnn.mobile.android.phone");
startActivity(i);
```

## Implicit Intents
- It can be cumbersome to list out all possible third-party apps you want to interact with
    - Not to mention very unrobust!
- It is better to use an implicit intent
    - Same intent object, created a different way
    - When you send the intent the OS will determine which app to open

## Parts of an Implicit Intent
- The Action
    - The most imporant part, all apps responding to implicit intents must indicate one or more activities
- The Category
    - Provides additional information about how the application can be used, for example,
        - `android.intent.category.BROWSABLE` - Allows your app to respond to links as well

## Parts of an Implicit Intent
- The Data
    - Type
        - Provides additional information about the type of data, so that a video playing app will only respond to intents involving video data
    - Scheme
        - Can restrict to certain schemes, ie only SMS or MMS

## Creating the Implicit Intent
- The intent constructor for an implicit intent takes in an activity, and optionally a data URI
```java
Intent implicit = new Intent(String ACTION_CONSTANT);
```
OR
```java
Intent implicit = new Intent(String ACTION_CONSTANT, URI Data);
```

## Creating the Implicit Intent
- The Intent object can be further refined by specifying the category and data type
- Category 
    - Can have mulitple
```java
implicitIntent.addCategory(String CATEGORY_CONSTANT);
```
- DataType
    - Can only have one
    - Can sometimes be infered from URI
```java
implicitIntent.setType(String dataType);
```

## Common Intents
- A list of common needs and their intents can be found at https://developer.android.com/guide/components/intents-common.html
- Two common ones are 
    - `Intent.ACTION_VIEW`
    - `Intent.ACTION_SEND`

## Resolving the Intent
- The OS find the correct activity for us when we use an implicit intent
    - What is there is no correct activity?
- Prior to sending an implicit intent, it is good practice to check if there is at least one applicaton that can handle it
- This is done through the method `resolveActivity` on the intent itself
    - It is passed a package manager (`getPackageManager`)

## Getting a Picture Back
- Just as before, an implicit intent can be launched with the method `startActivityForResult`
- A common use of this is to use the built in camera to get a picture
    - The action is `MediaStore.ACTION_CAPTURE_PHOTO`
    - There is no special category or data type needed
    - A URI should be placed in the extra EXTRA_OUTPUT, indicating where to save the photo
        - We can't do this until we talk about permissions later
    - A thumbnail is available in the `data` extra as a `Bitmap` object

## Sharing With Other Apps
- You can write your own share function, using ACTION_SEND and some other default settings
- This is so common that there is a built-in builder to create the correct intent for you
```java
ShareCompat.IntentBuilder
    .from(this)         // information about the calling activity
    .setType(mimeType)  // mime type for the data
    .setChooserTitle("Share this text with: ") //title for the app chooser
    .setText(txt)       // intent data
    .startChooser(); //launch chooser
```
<small>Example from https://google-developer-training.gitbooks.io/android-developer-fundamentals-course-concepts/content/en/Unit%201/23_c_activities_and_implicit_intents.html#sharingdatawithsharecompat</small>

## Manifest
- How does the OS know which apps can handle which intents?
    - From the AndroidManifest.XML file
- The manifest describes all parts of the app
    - Which activity is the main activity
    - Should app data be backed up
    - The intents each activity can handle
    - The permissions needed by the app 

## Intent Filters
- The intents handled by an activity are given in the AndroidManifest.xml file as an IntentFilter
```xml
<manifest>
    <application>
        <activity>
            <intent-filter>
                <action />
                <category />
                <data />
            </intent-filter>
        </activity>
    </application>
</manifest>
```

## Intent Filters
- The action has one attribute
    - android:name - Set to the fully qualified ACTION constant
- The category has one attribute
    - android:name - Set to the full qualified CATEGORY constant
    - To recieve any intents, you must have the default category set
        - `android.intent.category.DEFAULT`
- The data tag has numerous attributes
    - `android:mimeType`
    - `android:scheme`
    - `android:path`

## Responding to an Implicit Intent
- Inside the Activity, call `getIntent()` to recieve how this activity was started
    - What method should this be placed in?
- From this point on, there is no difference between handling an implicit and explicit intent
    - If you want to handle them differently, you can check various properties of the intent, such as packageName
